In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.runnables import RunnableSequence, RunnablePassthrough, RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
import os

load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")
api_key_02 = os.getenv("GOOGLE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01,
    temperature=0.1 # Lower temperature for better JSON generation
)

model_01 = ChatHuggingFace(llm=llm) 

model_02 = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.1, api_key=api_key_02)  

str_parser = StrOutputParser()

prompt_01 = PromptTemplate(
    template="Write a joke relevant to the following topic: {topic}",
    input_variables=["topic"]
)

prompt_02 = PromptTemplate(
    template="Expin the following joke: {joke}",
    input_variables=["joke"]
)

def count_words(text: str):
    return len(text.split())

joke_gen_chain = RunnableSequence(
    prompt_01 | model_01 | str_parser
)

parallel_chain = RunnableParallel(
    {
        'joke': RunnablePassthrough(),
        'word_count': RunnableLambda(count_words)
    }
)

final_chain = RunnableSequence(
    joke_gen_chain | parallel_chain
)

result = final_chain.invoke({"topic": "Black Hole"})

final_result = """{} \n\nword count: {}""".format(result['joke'], result['word_count'])

print(final_result)

Why did the astronaut break up with his girlfriend before going near the black hole?

Because he realized he was in a gravitational relationship that was sucking the life out of him. 

word count: 32


In [ ]:
final_chain.get_graph().print_ascii()

               +-------------+                 
               | PromptInput |                 
               +-------------+                 
                      *                        
                      *                        
                      *                        
              +----------------+               
              | PromptTemplate |               
              +----------------+               
                      *                        
                      *                        
                      *                        
             +-----------------+               
             | ChatHuggingFace |               
             +-----------------+               
                      *                        
                      *                        
                      *                        
             +-----------------+               
             | StrOutputParser |               
             +-----------------+        